In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from squlearn.feature_map import ChebRx
from squlearn.expectation_operator import SummedPaulis
from squlearn.qnn.loss import SquaredLoss
from squlearn.qnn.qnnc import QNNClassifier
from squlearn.optimizers import SLSQP
from squlearn.util import Executor

In [ ]:
executor = Executor("statevector_simulator")

Define parameterized quantum circuit $Q(x,\theta)$

In [ ]:
nqubits = 4

number_of_layers = 2
pqc = ChebRx(nqubits, 2, num_layers=number_of_layers)
pqc.draw()

Definition of the cost operator:

In [ ]:
cost_op = SummedPaulis(nqubits)
print(cost_op)

In [ ]:
np.random.seed(24)
param_ini = np.random.rand(pqc.num_parameters)
param_op_ini = np.random.rand(cost_op.num_parameters)

clf = QNNClassifier(pqc, cost_op, executor, SquaredLoss, SLSQP(), param_ini, param_op_ini)

### Generate Dataset

In [ ]:
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

X, y = make_blobs(60, centers=2, random_state=0)
X = MinMaxScaler((-0.9, 0.9)).fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

Optimization of the classifier

In [ ]:
clf.fit(X_train, y_train)

In [ ]:
print("Result Param:", clf.param)
print("Result ParamOp:", clf.param_op)

In [ ]:
clf.score(X_test, y_test)

In [ ]:
from matplotlib.colors import ListedColormap

xx, yy = np.meshgrid(np.arange(-0.99, 0.99, 0.01), np.arange(-0.99, 0.99, 0.01))

cm = plt.cm.RdBu
cm_bright = ListedColormap(["#FF0000", "#0000FF"])

Z = clf.predict_proba(np.column_stack([xx.ravel(), yy.ravel()]))[:, 1]
Z = Z.reshape(xx.shape)
plt.contourf(xx, yy, Z, cmap=cm, alpha=0.6)

plt.scatter(X_train[:, 0], X_train[:, 1], c=y_train, cmap=cm_bright)
# and testing points
plt.scatter(X_test[:, 0], X_test[:, 1], c=y_test, cmap=cm_bright, alpha=0.6, marker="X")